In [3]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-18 04:08:47--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-04-18 04:08:49 (1.21 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [7]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3DT59XH7HXR9K|          5|            0|          0|   N|                Y|
|R1LFS11BNASSU8|          5|            0|          1|   N|                Y|
|R296RT05AG0AF6|          5|            0|          0|   N|                Y|
|R3V37XDZ7ZCI3L|          5|            0|          1|   N|                Y|
|R14GU232NQFYX2|          5|            0|          0|   N|                Y|
| RZQH4V7L2O1PL|          1|            1|          1|   N|                Y|
|R1F8JMOSPJ3KO7|          5|            0|          0|   N|                Y|
|R1ZISGY2BWW4Z5|          5|            0|          0|   N|                Y|
|R17PW4I3AE5WZW|          5|            0|          0|   N|                Y|
|R3D93G1KTP6A8P|          3|            0|          0|   N|     

In [9]:
# Filter the dataframe to include only rows where total votes are >= 20
vine_filtered_df = vine_df.filter(df["total_votes"] >= 20)

# Create a new dataframe where the proportion of helpful votes to total votes is >= 50%
vine_helpful_df = vine_filtered_df.filter(
    vine_filtered_df["helpful_votes"] / vine_filtered_df["total_votes"] >= 0.5)

# Create a new dataframe containing only reviews part of the Vine (paid) program
vine_paid_df = vine_helpful_df.filter(vine_helpful_df["vine"] == 'Y')

# Create a new dataframe containing only reviews NOT part of the Vine (paid) program
vine_unpaid_df = vine_helpful_df.filter(vine_helpful_df["vine"] == 'N')

# Determine total # of reviews
total_review_count = vine_paid_df.count() + vine_unpaid_df.count()
total_paid_reviews = vine_paid_df.count()
total_unpaid_reviews = vine_unpaid_df.count()

# Determine total paid/unpaid five star reviews
total_paid_five_star_reviews = vine_paid_df.filter(vine_paid_df['star_rating'] == 5).count()
total_unpaid_five_star_reviews = vine_unpaid_df.filter(vine_unpaid_df['star_rating'] == 5).count()

# Determine proportion of paid/unpaid five star reviews
percent_paid_five_star_reviews = total_paid_five_star_reviews / total_paid_reviews
percent_unpaid_five_star_reviews = total_unpaid_five_star_reviews / total_unpaid_reviews



KeyboardInterrupt: ignored